# Set up environment

In [2]:
#pip install pandas scipy matplotlib imageio cv2 opencv-python torch pyyaml tqdm torchvision seaborn

In [1]:
# import relevant libraries
from IPython.display import Image
import pandas as pd
import random
import os
import glob
import shutil
from scipy import ndimage
import matplotlib.pyplot as plt
import imageio
import json
import numpy as np
import cv2

In [2]:
%cd ../6_train_yolo_four_class_tooth_classification

C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification


In [3]:
# remove yolov5 folder
# !rm -r 'C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/yolov5'

In [4]:
# clone repo
if os.path.exists('C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/yolov5/') == False:
    !git clone https://github.com/ultralytics/yolov5
    %cd yolov5
else:
    %cd yolov5
    !git init
    !git pull https://github.com/ultralytics/yolov5

C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\yolov5
Reinitialized existing Git repository in C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/yolov5/.git/
Already up to date.


From https://github.com/ultralytics/yolov5
 * branch            HEAD       -> FETCH_HEAD


In [5]:
# install dependencies
if os.path.exists('C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/yolov5/') == False:
    !pip install -r yolov5/requirements.txt

In [6]:
import torch
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.9.1+cu102 _CudaDeviceProperties(name='Quadro RTX 8000', major=7, minor=5, total_memory=49152MB, multi_processor_count=72)


# Set up data

In [7]:
# remote data folder
#!rm -r 'C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/data'
#!rm -r 'C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/unseen_images'

In [8]:
# prepare data directory structure for Yolo
path = 'C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/'

if os.path.exists(path + 'data/') == False:
    os.mkdir(path + 'data/')
    os.mkdir(path + 'data/images/')
    os.mkdir(path + 'data/images/test')
    os.mkdir(path + 'data/images/train')
    os.mkdir(path + 'data/images/valid/')
    os.mkdir(path + 'data/labels/')
    os.mkdir(path + 'data/labels/test')
    os.mkdir(path + 'data/labels/train')
    os.mkdir(path + 'data/labels/valid/')
    os.mkdir(path + 'unseen_images/')

In [11]:
# define variables
path = 'C:/Users/dental-1/Documents/dent_seg/'
folder = '6_train_yolo_four_class_tooth_classification/'
bounding_boxes = os.listdir(path + folder + 'bounding_boxes')
images_jpg = glob.glob(path + '2_dental_images/raw_images/*.jpg')
image_png = glob.glob(path + '2_dental_images/raw_images/*.png')
random.seed(1234)
norm = np.zeros((800,800))
image_names = []

for img in images_jpg:
    image_names.append(img.split('\\')[-1].split('.')[0])
    
for img in image_png:
    image_names.append(img.split('\\')[1][:-4])

In [12]:
# define function to organise data
def setup_data(bounding_boxes):   
    for bb in bounding_boxes:
        if 'cate' in bb:
            file_type = '.jpg'
        else:
            file_type = '.png'
        if bb.split('.txt')[0] in image_names:
            image_path = path + '2_dental_images/raw_images/' + bb.split('.txt')[0] + file_type
            source = path + folder + 'bounding_boxes/' + bb
            # normalise image
            unnorm_img = imageio.imread(image_path, as_gray=True)
            normalized_img = cv2.normalize(unnorm_img, norm, 0, 255, cv2.NORM_MINMAX)
            if random.random() > 0.3:
                destination = path + folder + 'data/labels/train/' + bb
                shutil.copy(source, destination)
                # copy image to its respective folder
                cv2.imwrite(path + folder + 'data/images/train/' + bb.split('.txt')[0] + file_type, normalized_img)
            else:
                # assign test data to proper location
                destination_test = path + folder + 'data/labels/test/' + bb
                shutil.copy(source, destination_test)
                # copy image to its respective folder
                cv2.imwrite(path + folder + 'data/images/test/' + bb.split('.txt')[0] + file_type, normalized_img)
                # validation housekeeping
                destination_val = path + folder + 'data/labels/valid/' + bb
                shutil.copy(source, destination_val)
                # copy image to its respective folder
                cv2.imwrite(path + folder + 'data/images/valid/' + bb.split('.txt')[0] + file_type, normalized_img)
            image_names.remove(bb.split('.txt')[0]) 
    for image in image_names:
        if 'cate' in image:
            image_path = path + '2_dental_images/raw_images/' + image + '.jpg'
        else:
            image_path = path + '2_dental_images/raw_images/' + image + '.png'
        # normalise image
        unnorm_img = imageio.imread(image_path, as_gray=True)
        normalized_img = cv2.normalize(unnorm_img, norm, 0, 255, cv2.NORM_MINMAX)
        # copy image to its respective folder
        if 'cate' in image:
            cv2.imwrite(path + folder + 'unseen_images/' + image +'.jpg', normalized_img)
        else:
            cv2.imwrite(path + folder + 'unseen_images/' + image +'.png', normalized_img)

In [13]:
# populate folders
if len(os.listdir(path + folder + '/data/images/test')) <= 1:
    setup_data(bounding_boxes)

# Configure YAML files

In [14]:
with open('model_yaml', 'w+') as file:
    file.write(
        """
        # parameters
        nc: 4  # number of classes
        depth_multiple: 0.33  # model depth multiple
        width_multiple: 0.50  # layer channel multiple

        # anchors
        anchors:
          - [10,13, 16,30, 33,23]  # P3/8
          - [30,61, 62,45, 59,119]  # P4/16
          - [116,90, 156,198, 373,326]  # P5/32

        # YOLOv5 backbone
        backbone:
          # [from, number, module, args]
          [[-1, 1, Focus, [64, 3]],  # 0-P1/2
           [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
           [-1, 3, BottleneckCSP, [128]],
           [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
           [-1, 9, BottleneckCSP, [256]],
           [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
           [-1, 9, BottleneckCSP, [512]],
           [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
           [-1, 1, SPP, [1024, [5, 9, 13]]],
           [-1, 3, BottleneckCSP, [1024, False]],  # 9
          ]

        # YOLOv5 head
        head:
          [[-1, 1, Conv, [512, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 6], 1, Concat, [1]],  # cat backbone P4
           [-1, 3, BottleneckCSP, [512, False]],  # 13

           [-1, 1, Conv, [256, 1, 1]],
           [-1, 1, nn.Upsample, [None, 2, 'nearest']],
           [[-1, 4], 1, Concat, [1]],  # cat backbone P3
           [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

           [-1, 1, Conv, [256, 3, 2]],
           [[-1, 14], 1, Concat, [1]],  # cat head P4
           [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

           [-1, 1, Conv, [512, 3, 2]],
           [[-1, 10], 1, Concat, [1]],  # cat head P5
           [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

           [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
          ]
        """
    )

In [15]:
with open('data_yaml', 'w+') as file:
    file.write(
        """
        train: C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/data/images/train
        val: C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/data/images/valid
        test: C:/Users/dental-1/Documents/dent_seg/6_train_yolo_four_class_tooth_classification/data/images/test

        nc: 4
        names: ['molar', 'premolar', 'canine', 'incisor']
        """
    )

# Train model

In [7]:
%%time
!python train.py --batch 16 --epochs 1500 --data data_yaml --cfg model_yaml --weights ../../4_weights/tooth_segmentation_weight.pt  --name teeth_classification --cache

github: up to date with https://github.com/ultralytics/yolov5 Wall time: 16min 39s
Weights & Biases: run 'pip install wandb' to automatically track and visualize YOLOv5  runs (RECOMMENDED)
module 'signal' has no attribute 'SIGALRM'
Optimizer stripped from runs\train\teeth_classification4\weights\last.pt, 14.9MB
Optimizer stripped from runs\train\teeth_classification4\weights\best.pt, 14.9MB



train: weights=../../4_weights/tooth_segmentation_weight.pt, cfg=model_yaml, data=data_yaml, hyp=data\hyps\hyp.scratch.yaml, epochs=1500, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs\train, name=teeth_classification, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  v6.0-103-g7a39803 torch 1.9.1+cu102 CUDA:0 (Quadro RTX 8000, 49152MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shea


     Epoch   gpu_mem       box       obj       cls    labels  img_size

  0%|          | 0/9 [00:00<?, ?it/s]                                           
    4/1499      2.5G   0.09426     0.349   0.04204       888       640:   0%|   
    4/1499      2.5G   0.09426     0.349   0.04204       888       640:  11%|#1 
    4/1499      2.5G   0.09266    0.3297   0.04219       748       640:  11%|#1 
    4/1499      2.5G   0.09266    0.3297   0.04219       748       640:  22%|##2
    4/1499      2.5G   0.09323    0.3519    0.0421      1036       640:  22%|##2
    4/1499      2.5G   0.09323    0.3519    0.0421      1036       640:  33%|###
    4/1499      2.5G   0.09188    0.3363   0.04219       658       640:  33%|###
    4/1499      2.5G   0.09188    0.3363   0.04219       658       640:  44%|###
    4/1499      2.5G   0.09152    0.3305    0.0422       739       640:  44%|###
    4/1499      2.5G   0.09152    0.3305    0.0422       739       640:  56%|###
    4/1499      2.5G   0.09112    0.

   29/1499      2.5G   0.05488    0.1303   0.03548       607       640:  67%|###
   29/1499      2.5G   0.05488    0.1303   0.03548       607       640:  78%|###
   29/1499      2.5G   0.05485    0.1336   0.03543       945       640:  78%|###
   29/1499      2.5G   0.05485    0.1336   0.03543       945       640:  89%|###
   29/1499      2.5G   0.05465    0.1353    0.0356        94       640:  89%|###
   29/1499      2.5G   0.05465    0.1353    0.0356        94       640: 100%|###

               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         58       1608      0.277      0.611      0.417      0.252

     Epoch   gpu_mem       box       obj       cls    labels  img_size

  0%|          | 0/9 [00:00<?

               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         58       1608      0.488      0.841      0.704      0.541

     Epoch   gpu_mem       box       obj       cls    labels  img_size

  0%|          | 0/9 [00:00<?, ?it/s]                                           
  121/1499      2.5G   0.03942    0.1132   0.02085       839       640:   0%|   
  121/1499      2.5G   0.03942    0.1132   0.02085       839       640:  11%|#1 
  121/1499      2.5G   0.03794    0.1061    0.0208       721       640:  11%|#1 
  121/1499      2.5G   0.03794    0.1061    0.0208       721       640:  22%|##2
  121/1499      2.5G   0.03756    0.1079   0.02101       806       640:  22%|##2
  121/1499      2.5G   0.03756

  245/1499      2.5G   0.03433    0.1095  0.009456       815       640:   0%|   
  245/1499      2.5G   0.03433    0.1095  0.009456       815       640:  11%|#1 
  245/1499      2.5G   0.03418    0.1014  0.008818       654       640:  11%|#1 
  245/1499      2.5G   0.03418    0.1014  0.008818       654       640:  22%|##2
  245/1499      2.5G   0.03564    0.1065  0.009099       932       640:  22%|##2
  245/1499      2.5G   0.03564    0.1065  0.009099       932       640:  33%|###
  245/1499      2.5G   0.03631    0.1082  0.009212       901       640:  33%|###
  245/1499      2.5G   0.03631    0.1082  0.009212       901       640:  44%|###
  245/1499      2.5G   0.03678    0.1127  0.009265       987       640:  44%|###
  245/1499      2.5G   0.03678    0.1127  0.009265       987       640:  56%|###
  245/1499      2.5G   0.03675    0.1137  0.009153       900       640:  56%|###
  245/1499      2.5G   0.03675    0.1137  0.009153       900       640:  67%|###
  245/1499      2.5G   0.036

  369/1499      2.5G   0.03189   0.09725  0.006001       721       640:  78%|###
  369/1499      2.5G   0.03175   0.09785  0.005953       766       640:  78%|###
  369/1499      2.5G   0.03175   0.09785  0.005953       766       640:  89%|###
  369/1499      2.5G   0.03282    0.1003  0.005977       121       640:  89%|###
  369/1499      2.5G   0.03282    0.1003  0.005977       121       640: 100%|###
  369/1499      2.5G   0.03282    0.1003  0.005977       121       640: 100%|###

               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         58       1608      0.947      0.952      0.971      0.785

     Epoch   gpu_mem       box       obj       cls    labels  img_size

  0%|          | 0/9 [00:00<?

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         58       1608      0.955      0.931      0.964      0.771

     Epoch   gpu_mem       box       obj       cls    labels  img_size

  0%|          | 0/9 [00:00<?, ?it/s]                                           
  494/1499      2.5G   0.03043   0.08349  0.005276       723       640:   0%|   
  494/1499      2.5G   0.03043   0.08349  0.005276       723       640:  11%|#1 
  494/1499      2.5G   0.03116   0.09029  0.005157       875       640:  11%|#1 
  494/1499      2.5G   0.03116   0.09029  0.005157       875       640:  22%|##2
  494/1499      2.5G   0.03092   0.09382  0.005335       889       640:  22%|##2
  494/1499      2.5G   0.03092   0.09382  0.005335       889       640:  33%|###
  494/1499      2.5G   0.03106   0.09505  0.005569       867       640:  33%|###
  494/1499      2.5G   0.03106   0.09505  0.005569       867       640:  44%|###
  494/1499      2.5G   0.03099

# Test model

In [7]:
# run model again test image set
!python detect.py --agnostic --weights runs/train/teeth_classification2/weights/best.pt --conf 0.4 --source ../data/images/test/ --name test_label --save-txt --save-conf

detect: weights=['runs/train/teeth_classification2/weights/best.pt'], source=../data/images/test/, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=True, augment=False, visualize=False, update=False, project=runs\detect, name=test_label, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.0-100-g5185981 torch 1.9.1+cu102 CUDA:0 (Quadro RTX 8000, 49152MiB)

Fusing layers... 
Model Summary: 232 layers, 7254609 parameters, 0 gradients
image 1/58 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\data\images\test\cate1-00004.jpg: 384x640 12 molars, 7 premolars, 5 canines, 8 incisors, Done. (0.000s)
image 2/58 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\data\images\test\cate1-00009.jpg: 384x640 12 molars, 8 premolars, 4 canines, 8 incisors, Done.

In [8]:
# run model against images in unseen image folder
!python detect.py --agnostic --weights runs/train/teeth_classification2/weights/best.pt --conf 0.4 --source ../unseen_images/ --name unseen_label --save-txt --save-conf

detect: weights=['runs/train/teeth_classification2/weights/best.pt'], source=../unseen_images/, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=True, augment=False, visualize=False, update=False, project=runs\detect, name=unseen_label, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.0-100-g5185981 torch 1.9.1+cu102 CUDA:0 (Quadro RTX 8000, 49152MiB)

Fusing layers... 
Model Summary: 232 layers, 7254609 parameters, 0 gradients
image 1/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.1.1637304243952960394983328788351918369.png: 384x640 7 molars, 9 premolars, 3 canines, 8 incisors, Done. (0.000s)
image 2/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.1.163730424399089668543

image 495/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.3.6.23645617616308960458410687002031817859154.png: 352x640 8 molars, 9 premolars, 3 canines, 6 incisors, Done. (0.016s)
image 496/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.3.6.23651744768105250458700386057520261137780.png: 352x640 8 molars, 8 premolars, 4 canines, 8 incisors, Done. (0.000s)
image 497/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.3.6.23663814688010380458346367063414664508307.png: 352x640 9 molars, 7 premolars, 3 canines, 2 incisors, Done. (0.016s)
image 498/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.3.6.23676527104576490458432498279342370907883.png: 320x640 8 molars, 4 premolars, 3 canines, 7 incisors, Done. (0.016s)
image 499/2310 C

image 995/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.3.6.33010757904331160458561279037504241346185.png: 352x640 12 molars, 7 premolars, 4 canines, 9 incisors, Done. (0.000s)
image 996/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.3.6.33014659440040290458738430036944038406997.png: 352x640 8 molars, 8 premolars, 4 canines, 8 incisors, Done. (0.016s)
image 997/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.3.6.33015400384460090458515507208984473402628.png: 384x640 8 molars, 7 premolars, 5 canines, 8 incisors, Done. (0.016s)
image 998/2310 C:\Users\dental-1\Documents\dent_seg\6_train_yolo_four_class_tooth_classification\unseen_images\1.2.840.114257.3.6.33046354448204560458501119076482856212679.png: 352x640 10 molars, 9 premolars, 3 canines, 8 incisors, Done. (0.000s)
image 999/2310

In [ ]:
# run model against specific image
!python detect.py --agnostic --weights runs/train/teeth_classification2/weights/best.pt --conf 0.4 --source ../../2_dental_images/raw_images/cate1-00001.jpg --save-txt --save-conf